In [20]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model(
    "spectrogram_autoencoder_section00.h5",
    custom_objects={"mse": tf.keras.losses.MeanSquaredError()}
)


In [21]:
dummy_input = tf.zeros((1, 224, 224, 1))
model(dummy_input)


<tf.Tensor: shape=(1, 224, 224, 1), dtype=float32, numpy=
array([[[[2.40558520e-01],
         [1.65248230e-01],
         [1.33338124e-01],
         ...,
         [2.63283346e-02],
         [4.08077948e-02],
         [1.44534871e-01]],

        [[1.71662197e-01],
         [9.37700942e-02],
         [7.17761219e-02],
         ...,
         [4.26338147e-03],
         [1.21996645e-02],
         [7.00060725e-02]],

        [[1.40339553e-01],
         [7.65174031e-02],
         [5.08002453e-02],
         ...,
         [2.40243692e-03],
         [9.35547892e-03],
         [5.53014949e-02]],

        ...,

        [[2.93886550e-02],
         [4.73229960e-03],
         [2.21101986e-03],
         ...,
         [2.99242515e-06],
         [3.70032540e-05],
         [2.50112871e-03]],

        [[4.65153903e-02],
         [1.82750206e-02],
         [8.80100485e-03],
         ...,
         [9.02277170e-05],
         [7.72390224e-04],
         [1.03171514e-02]],

        [[1.61471024e-01],
         [9

In [23]:
import tensorflow as tf

input_img = tf.keras.Input(shape=(224, 224, 1))


In [33]:
input_img = tf.keras.Input(shape=(224, 224, 1))

x = model.layers[0](input_img)  # conv
x = model.layers[1](x)          # pool

x = model.layers[2](x)
x = model.layers[3](x)

x = model.layers[4](x)
encoder_output = model.layers[5](x)   # <-- STOP HERE (28×28×128)

encoder = tf.keras.Model(input_img, encoder_output)
encoder.summary()


Model: "functional_81"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,672 (362.00 KB)

 Trainable params: 92,672 (362.00 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
import os
import numpy as np

def extract_features(folder_path, encoder):
    features = []
    filenames = []

    for file in os.listdir(folder_path):
        if file.endswith(".png") or file.endswith(".jpg"):
            img_path = os.path.join(folder_path, file)

            img = load_image(img_path)              # same function you already use
            img = np.expand_dims(img, axis=0)       # (1, 224, 224, 1)

            feat = encoder.predict(img, verbose=0)
            feat = feat.flatten()                   # VERY IMPORTANT

            features.append(feat)
            filenames.append(file)

    return np.array(features), filenames


In [27]:
TRAIN_DIR = r"C:\Users\praka\OneDrive\Desktop\Machine Learning\Deep-Learning\ImageToSpectrogramPrediction\SpectrogramImage\train"

train_features, _ = extract_features(TRAIN_DIR, encoder)

print("Train feature shape:", train_features.shape)


Train feature shape: (1004, 401408)


In [28]:
centroid = np.mean(train_features, axis=0)
print("Centroid shape:", centroid.shape)


Centroid shape: (401408,)


In [29]:
SOURCE_TEST_DIR=r"C:\Users\praka\OneDrive\Desktop\Machine Learning\Deep-Learning\ImageToSpectrogramPrediction\SpectrogramImage\source_test"
TARGET_TEST_DIR=r"C:\Users\praka\OneDrive\Desktop\Machine Learning\Deep-Learning\ImageToSpectrogramPrediction\SpectrogramImage\target_test"

In [30]:
source_features, source_files = extract_features(SOURCE_TEST_DIR, encoder)
target_features, target_files = extract_features(TARGET_TEST_DIR, encoder)

print("Source features:", source_features.shape)
print("Target features:", target_features.shape)


Source features: (300, 401408)
Target features: (204, 401408)


In [31]:
from numpy.linalg import norm

source_scores = np.array([norm(f - centroid) for f in source_features])
target_scores = np.array([norm(f - centroid) for f in target_features])


In [32]:
source_normal_scores = []
source_anomaly_scores = []

for score, fname in zip(source_scores, source_files):
    if "normal" in fname.lower():
        source_normal_scores.append(score)
    elif "anomaly" in fname.lower():
        source_anomaly_scores.append(score)

print("SOURCE TEST")
print("Mean normal distance:", np.mean(source_normal_scores))
print("Mean anomaly distance:", np.mean(source_anomaly_scores))


SOURCE TEST
Mean normal distance: 133.8014
Mean anomaly distance: 132.9808


In [34]:
def normalize_features(features):
    norm = np.linalg.norm(features, axis=1, keepdims=True)
    return features / (norm + 1e-10)


In [35]:
train_features, _ = extract_features(TRAIN_DIR, encoder)
train_features = normalize_features(train_features)

print(train_features.shape)


(1004, 100352)


In [36]:
centroid = np.mean(train_features, axis=0)
centroid = centroid / (np.linalg.norm(centroid) + 1e-10)


In [37]:
source_features, source_files = extract_features(SOURCE_TEST_DIR, encoder)
target_features, target_files = extract_features(TARGET_TEST_DIR, encoder)

source_features = normalize_features(source_features)
target_features = normalize_features(target_features)


In [38]:
def cosine_distance(features, centroid):
    return 1 - np.dot(features, centroid)


In [39]:
source_scores = cosine_distance(source_features, centroid)
target_scores = cosine_distance(target_features, centroid)


In [40]:
source_normal, source_anomaly = [], []

for s, f in zip(source_scores, source_files):
    if "normal" in f.lower():
        source_normal.append(s)
    elif "anomaly" in f.lower():
        source_anomaly.append(s)

print("SOURCE TEST")
print("Mean normal distance:", np.mean(source_normal))
print("Mean anomaly distance:", np.mean(source_anomaly))


SOURCE TEST
Mean normal distance: 0.0017452668
Mean anomaly distance: 0.0019599113


In [41]:
threshold_source = np.percentile(source_normal, 95)
print("Source threshold:", threshold_source)


Source threshold: 0.0038483024


In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(scores, files, threshold):
    y_true = []
    y_pred = []

    for s, f in zip(scores, files):
        if "normal" in f.lower():
            y_true.append(0)
        elif "anomaly" in f.lower():
            y_true.append(1)

        y_pred.append(1 if s > threshold else 0)

    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred),
        "ROC_AUC": roc_auc_score(y_true, scores)
    }

print("SOURCE TEST METRICS")
print(evaluate(source_scores, source_files, threshold_source))


SOURCE TEST METRICS
{'Accuracy': 0.6433333333333333, 'Precision': 0.4444444444444444, 'Recall': 0.0761904761904762, 'F1': 0.13008130081300814, 'ROC_AUC': np.float64(0.5701587301587302)}
